<a href="https://www.kaggle.com/code/ryancardwell/orcastratorv1-1?scriptVersionId=271493342" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
#   Cell 1
import torch
import torch.nn as nn
import numpy as np
import json
import time
import logging
from functools import lru_cache
from collections import defaultdict, deque
from scipy import ndimage
import math
from typing import List, Dict, Tuple, Optional, Any

# Enhanced configuration for S-tier performance
class ARCConfig:
    def __init__(self):
        # Extended runtime (5x from user)
        self.MAX_BEAM_DEPTH = 12
        self.MAX_BEAM_WIDTH = 64
        self.MAX_RUNTIME_PER_TASK = 300
        self.MAX_PROGRAM_LENGTH = 20
        
        # 3D projection enhancements
        self.USE_3D_EARLY = True
        self.USE_3D_ON_HARD = True
        self.MAX_3D_DEPTH = 8
        self.ENABLE_3D_PATTERNS = True
        
        # Enhanced search strategies
        self.DYNAMIC_BEAM_SCALING = True
        self.ADAPTIVE_DEPTH = True
        self.MULTI_PASS_SEARCH = True
        self.EARLY_TERMINATION_PATIENCE = 5
        
        # Neural and caching
        self.VETO_THRESHOLD = 0.25
        self.LRU_CACHE_SIZE = 5000
        self.PRUNE_EARLY = True
        
        # Novel insights integration
        self.FUZZY_ALPHA = 0.5
        self.QUANTUM_DIMS = 4
        self.HYPER_DIM = 128
        self.CONFIDENCE_DECAY = 0.95
        
        # Original optimized parameters
        self.SEED = 1337
        self.BG = 0
        self.MAX_GRID_SIDE = 60
        self.ATTEMPTS_PER_TEST = 2

config = ARCConfig()

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Enhanced logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Core utilities with 3D support
def grid_to_numpy(grid):
    return np.array(grid, dtype=np.int32)

def numpy_to_grid(arr):
    return arr.tolist()

@lru_cache(maxsize=config.LRU_CACHE_SIZE)
def calculate_iou(grid1, grid2):
    if isinstance(grid1, list):
        grid1 = grid_to_numpy(grid1)
    if isinstance(grid2, list):
        grid2 = grid_to_numpy(grid2)
    
    intersection = np.sum((grid1 > 0) & (grid2 > 0))
    union = np.sum((grid1 > 0) | (grid2 > 0))
    return intersection / union if union > 0 else 0.0

def extract_palette(grid):
    if isinstance(grid, list):
        grid = grid_to_numpy(grid)
    unique, counts = np.unique(grid, return_counts=True)
    return dict(zip(unique, counts))

# 3D Orthogonal Projection System (from user)
class OrthogonalProjector3D:
    def __init__(self):
        self.projection_cache = {}
        self.voxel_cache = {}
        
    def grid_to_3d_voxels(self, grid, depth_layers=4):
        cache_key = (tuple(map(tuple, grid)), depth_layers)
        if cache_key in self.voxel_cache:
            return self.voxel_cache[cache_key]
            
        if isinstance(grid, list):
            grid = grid_to_numpy(grid)
            
        h, w = grid.shape
        voxels = np.zeros((h, w, depth_layers), dtype=np.int32)
        
        for i in range(h):
            for j in range(w):
                color_val = grid[i, j]
                if color_val > 0:
                    depth = min(depth_layers - 1, color_val % depth_layers)
                    voxels[i, j, depth] = color_val
                    
        self.voxel_cache[cache_key] = voxels
        return voxels
    
    def project_3d_to_2d(self, voxels, projection_type='front'):
        cache_key = (voxels.tobytes(), projection_type)
        if cache_key in self.projection_cache:
            return self.projection_cache[cache_key]
            
        h, w, d = voxels.shape
        
        if projection_type == 'front':
            projection = np.max(voxels, axis=2)
        elif projection_type == 'top':
            projection = np.max(voxels, axis=0).T
        elif projection_type == 'side':
            projection = np.max(voxels, axis=1)
        elif projection_type == 'composite':
            front = np.max(voxels, axis=2)
            top = np.max(voxels, axis=0).T
            side = np.max(voxels, axis=1)
            max_h = max(front.shape[0], top.shape[0], side.shape[0])
            max_w = max(front.shape[1], top.shape[1], side.shape[1])
            projection = np.zeros((max_h, max_w), dtype=np.int32)
            projection[:front.shape[0], :front.shape[1]] = np.maximum(
                projection[:front.shape[0], :front.shape[1]], front)
        else:
            projection = np.max(voxels, axis=2)
            
        self.projection_cache[cache_key] = projection
        return projection

projector_3d = OrthogonalProjector3D()

# Enhanced NeuroBudget (from user)
class EnhancedNeuroBudget:
    def __init__(self, total_budget=1.0):
        self.total_budget = total_budget
        self.remaining_budget = total_budget
        self.task_budgets = {}
        self.complexity_scores = {}
        
    def allocate_task_budget(self, task_id, complexity_score):
        base_budget = 0.1
        complexity_factor = min(2.0, 1.0 + complexity_score)
        allocated = base_budget * complexity_factor
        self.task_budgets[task_id] = allocated
        return allocated
    
    def should_continue(self, task_id, elapsed_time):
        if task_id not in self.task_budgets:
            return True
        budget_used = elapsed_time / config.MAX_RUNTIME_PER_TASK
        return budget_used < self.task_budgets[task_id]

Using device: cpu


In [2]:
#  Cell 2
class EnhancedDSL:
    def __init__(self):
        self.primitives = {}
        self._register_primitives()
        self.dynamic_color_primitives = {}
        
    def _register_primitives(self):
        # Geometric transformations (optimized)
        self.primitives['rotate_90'] = lambda g: np.rot90(g, 1)
        self.primitives['rotate_180'] = lambda g: np.rot90(g, 2)
        self.primitives['rotate_270'] = lambda g: np.rot90(g, 3)
        self.primitives['flip_horizontal'] = lambda g: np.fliplr(g)
        self.primitives['flip_vertical'] = lambda g: np.flipud(g)
        self.primitives['transpose'] = lambda g: g.T
        
        # Color operations
        self.primitives['color_invert'] = self.color_invert
        self.primitives['color_replace'] = self.color_replace
        self.primitives['swap_most_common'] = self.swap_most_common
        
        # Size operations
        self.primitives['scale_2x'] = self.scale_2x
        self.primitives['scale_3x'] = self.scale_3x
        self.primitives['tile_2x2'] = lambda g: np.tile(g, (2, 2))
        self.primitives['crop_to_content'] = self.crop_to_content
        
        # Object operations
        self.primitives['extract_largest_object'] = self.extract_largest_object
        self.primitives['gravity_down'] = self.gravity_down
        
        # 3D projection operations (from user)
        self.primitives['project_3d_front'] = lambda g: projector_3d.project_3d_to_2d(
            projector_3d.grid_to_3d_voxels(g, config.MAX_3D_DEPTH), 'front')
        self.primitives['project_3d_top'] = lambda g: projector_3d.project_3d_to_2d(
            projector_3d.grid_to_3d_voxels(g, config.MAX_3D_DEPTH), 'top')
        self.primitives['project_3d_side'] = lambda g: projector_3d.project_3d_to_2d(
            projector_3d.grid_to_3d_voxels(g, config.MAX_3D_DEPTH), 'side')
        
        # Novel Insight 1: Adaptive scaling with fuzzy tuning
        self.primitives['adaptive_scale'] = lambda g, factor=2: np.repeat(np.repeat(g, factor, axis=0), factor, axis=1)
        
        # Novel Insight 2: Symmetry-aware operations
        self.primitives['symmetry_reflect'] = lambda g: np.maximum(g, np.fliplr(g))
        
        # Novel Insight 3: Density-based filling
        self.primitives['density_fill'] = lambda g: np.where(g == 0, np.mean(g[g > 0]) if np.any(g > 0) else 0, g)
        
        # Novel Insight 4: Pattern extrapolation
        self.primitives['pattern_extrapolate'] = self.pattern_extrapolate
        
        # Novel Insight 5: Causal inference modeling
        self.primitives['causal_infer'] = lambda g: np.cumsum(g, axis=0)
        
        # Novel Insight 6: Fuzzy thresholding
        self.primitives['fuzzy_threshold'] = lambda g: np.where(g > config.FUZZY_ALPHA * np.mean(g), g, 0)
        
        # Novel Insight 7: Quantum-inspired superposition
        self.primitives['quantum_superpose'] = self.quantum_superpose
        
        # Novel Insight 8: Bio-evolutionary adaptation
        self.primitives['bio_evolve'] = self.bio_evolve
        
        # Novel Insight 9: Multi-resolution analysis
        self.primitives['multi_resolution'] = self.multi_resolution
        
        # Novel Insight 10: Recursive synthesis
        self.primitives['recursive_synth'] = self.recursive_synth

    def generate_dynamic_color_primitives(self, train_colors):
        for color_in, color_out in train_colors:
            if color_in != color_out:
                prim_name = f'color_{color_in}_to_{color_out}'
                self.dynamic_color_primitives[prim_name] = \
                    lambda grid, cin=color_in, cout=color_out: self.color_replace(grid, cin, cout)
        self.primitives.update(self.dynamic_color_primitives)
    
    # Core operation implementations
    def color_invert(self, grid):
        inverted = grid.copy()
        mask = grid > 0
        if np.any(mask):
            max_color = np.max(grid[mask])
            inverted[mask] = max_color - grid[mask] + 1
        return inverted
    
    def color_replace(self, grid, color_from, color_to):
        replaced = grid.copy()
        replaced[grid == color_from] = color_to
        return replaced
    
    def swap_most_common(self, grid):
        if np.any(grid > 0):
            unique, counts = np.unique(grid[grid > 0], return_counts=True)
            if len(unique) >= 2:
                most_common = unique[np.argmax(counts)]
                second_common = unique[np.argsort(counts)[-2]]
                return self.color_replace(grid, most_common, second_common)
        return grid
    
    def scale_2x(self, grid):
        h, w = grid.shape
        scaled = np.zeros((h*2, w*2), dtype=grid.dtype)
        for i in range(h):
            for j in range(w):
                scaled[i*2:(i+1)*2, j*2:(j+1)*2] = grid[i, j]
        return scaled
    
    def scale_3x(self, grid):
        h, w = grid.shape
        scaled = np.zeros((h*3, w*3), dtype=grid.dtype)
        for i in range(h):
            for j in range(w):
                scaled[i*3:(i+1)*3, j*3:(j+1)*3] = grid[i, j]
        return scaled
    
    def crop_to_content(self, grid):
        if not np.any(grid > 0):
            return grid
        
        rows = np.any(grid > 0, axis=1)
        cols = np.any(grid > 0, axis=0)
        
        rmin, rmax = np.where(rows)[0][[0, -1]]
        cmin, cmax = np.where(cols)[0][[0, -1]]
        
        return grid[rmin:rmax+1, cmin:cmax+1]
    
    def extract_largest_object(self, grid):
        labeled, num_features = ndimage.label(grid > 0)
        if num_features == 0:
            return grid
        
        largest_component = None
        largest_size = 0
        for i in range(1, num_features + 1):
            component_mask = (labeled == i)
            size = np.sum(component_mask)
            if size > largest_size:
                largest_size = size
                largest_component = component_mask
        
        result = np.zeros_like(grid)
        result[largest_component] = grid[largest_component]
        return result
    
    def gravity_down(self, grid):
        result = np.zeros_like(grid)
        h, w = grid.shape
        
        for j in range(w):
            col = grid[:, j]
            non_zero = col[col > 0]
            if len(non_zero) > 0:
                result[h-len(non_zero):, j] = non_zero
        return result
    
    # Novel insight implementations
    def pattern_extrapolate(self, grid):
        h, w = grid.shape
        if h % 2 == 0:
            return np.tile(grid[:h//2], (2, 1))
        return grid
    
    def quantum_superpose(self, grid):
        # Quantum-inspired superposition of multiple transformations
        transformations = [
            lambda g: g,
            lambda g: np.rot90(g, 1),
            lambda g: np.fliplr(g),
            lambda g: np.flipud(g)
        ]
        weights = [0.4, 0.2, 0.2, 0.2]  # Fuzzy weights
        result = np.zeros_like(grid)
        
        for transform, weight in zip(transformations, weights):
            result = np.maximum(result, transform(grid) * weight)
        
        return np.clip(result, 0, 9).astype(int)
    
    def bio_evolve(self, grid, generations=3):
        # Bio-inspired evolutionary optimization
        current = grid.copy()
        for gen in range(generations):
            mutation = current + np.random.randint(-1, 2, size=current.shape)
            mutation = np.clip(mutation, 0, 9)
            
            # Fitness: prefer patterns with higher structure
            current_fitness = np.std(current)
            mutation_fitness = np.std(mutation)
            
            if mutation_fitness > current_fitness:
                current = mutation
        return current
    
    def multi_resolution(self, grid):
        # Multi-resolution analysis
        h, w = grid.shape
        if h > 4 and w > 4:
            # Downsample then upsample to capture patterns
            downsampled = grid[::2, ::2]
            upsampled = np.repeat(np.repeat(downsampled, 2, axis=0), 2, axis=1)
            # Blend with original
            return np.maximum(grid, upsampled[:h, :w])
        return grid
    
    def recursive_synth(self, grid, depth=2):
        # Recursive synthesis for complex patterns
        if depth == 0:
            return grid
        
        # Apply transformations recursively
        transformed = self.symmetry_reflect(grid)
        transformed = self.pattern_extrapolate(transformed)
        
        return self.recursive_synth(transformed, depth-1)

enhanced_dsl = EnhancedDSL()

In [3]:
#  Cell 3
class EnhancedTinyVetoNet(nn.Module):
    def __init__(self, input_dim=256, hidden_dim=128, num_layers=4):
        super().__init__()
        self.input_dim = input_dim
        
        layers = []
        layers.append(nn.Linear(input_dim, hidden_dim))
        layers.append(nn.ReLU())
        layers.append(nn.Dropout(0.2))
        
        for _ in range(num_layers - 2):
            layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(0.2))
            
        layers.append(nn.Linear(hidden_dim, 1))
        layers.append(nn.Sigmoid())
        
        self.network = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.network(x)
    
    def extract_features(self, grid):
        features = []
        features.extend(self._extract_2d_features(grid))
        
        if config.ENABLE_3D_PATTERNS:
            features.extend(self._extract_3d_features(grid))
            
        features.extend(self._extract_pattern_features(grid))
        
        # Ensure fixed dimension
        if len(features) < self.input_dim:
            features.extend([0] * (self.input_dim - len(features)))
        else:
            features = features[:self.input_dim]
            
        return torch.tensor(features, dtype=torch.float32).unsqueeze(0)
    
    def _extract_2d_features(self, grid):
        features = []
        features.append(np.mean(grid > 0))  # Density
        features.append(len(np.unique(grid[grid > 0])))  # Color variety
        
        h, w = grid.shape
        center_h, center_w = h // 2, w // 2
        center_region = grid[center_h-1:center_h+2, center_w-1:center_w+2]
        features.append(np.mean(center_region > 0))
        
        edges = np.concatenate([grid[0, :], grid[-1, :], grid[:, 0], grid[:, -1]])
        features.append(np.mean(edges > 0))
        
        return features
    
    def _extract_3d_features(self, grid):
        features = []
        try:
            voxels = projector_3d.grid_to_3d_voxels(grid, depth_layers=4)
            
            # Depth layer statistics
            for i in range(min(4, voxels.shape[2])):
                layer = voxels[:, :, i]
                features.append(np.mean(layer > 0))
                features.append(len(np.unique(layer[layer > 0])))
                
            # Symmetry features
            vertical_sym = np.mean(voxels == np.flip(voxels, axis=0))
            horizontal_sym = np.mean(voxels == np.flip(voxels, axis=1))
            features.extend([vertical_sym, horizontal_sym])
                
        except Exception as e:
            features.extend([0] * 10)
            
        return features
    
    def _extract_pattern_features(self, grid):
        features = []
        
        # Connected components analysis
        labeled, num_components = ndimage.label(grid > 0)
        features.append(num_components)
        if num_components > 0:
            sizes = [np.sum(labeled == i) for i in range(1, num_components + 1)]
            features.append(max(sizes))
            features.append(np.mean(sizes))
        else:
            features.extend([0, 0])
            
        # Row/column patterns
        row_patterns = [np.any(grid[i, :] > 0) for i in range(grid.shape[0])]
        col_patterns = [np.any(grid[:, j] > 0) for j in range(grid.shape[1])]
        features.append(np.mean(row_patterns))
        features.append(np.mean(col_patterns))
        
        return features

class EnhancedPolicyNetwork(nn.Module):
    def __init__(self, state_dim=256, num_primitives=50, hidden_dim=128):
        super().__init__()
        self.state_encoder = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )
        
        self.attention = nn.MultiheadAttention(hidden_dim, num_heads=4)
        self.primitive_predictor = nn.Linear(hidden_dim, num_primitives)
        
    def forward(self, state_features):
        encoded = self.state_encoder(state_features)
        attended, _ = self.attention(encoded.unsqueeze(0), encoded.unsqueeze(0), encoded.unsqueeze(0))
        logits = self.primitive_predictor(attended.squeeze(0))
        return torch.softmax(logits, dim=-1)

# Initialize neural components
enhanced_veto_net = EnhancedTinyVetoNet().to(device)
enhanced_policy_net = EnhancedPolicyNetwork().to(device)

In [4]:
#  Cell 4
class EnhancedBeamSearch:
    def __init__(self, dsl, config):
        self.dsl = dsl
        self.config = config
        self.cache = {}
        self.stats = defaultdict(int)
        
    def should_use_3d_early(self, task_complexity, grid_properties):
        if not self.config.USE_3D_EARLY:
            return False
            
        conditions = [
            task_complexity > 0.7 and self.config.USE_3D_ON_HARD,
            grid_properties.get('size', 0) > 100,
            grid_properties.get('sparsity', 0) > 0.8,
            grid_properties.get('density', 0) > 0.3,
        ]
        
        return any(conditions)
    
    def get_3d_enhanced_primitives(self, base_primitives, should_use_3d):
        enhanced = base_primitives.copy()
        
        if should_use_3d:
            enhanced_3d = [
                'project_3d_front', 'project_3d_top', 'project_3d_side',
                'voxelize_4layer', 'analyze_3d_patterns'
            ]
            for prim in reversed(enhanced_3d):
                if prim in enhanced:
                    enhanced.remove(prim)
                    enhanced.insert(0, prim)
        
        return enhanced
    
    def search(self, start_grid, target_grid, max_depth=None, max_width=None):
        if max_depth is None:
            max_depth = self.config.MAX_BEAM_DEPTH
        if max_width is None:
            max_width = self.config.MAX_BEAM_WIDTH
            
        # Analyze task complexity and grid properties
        task_complexity = self.analyze_task_complexity(start_grid, target_grid)
        grid_properties = self.analyze_grid_properties(start_grid)
        
        # Decide on 3D usage
        use_3d_early = self.should_use_3d_early(task_complexity, grid_properties)
        
        # Get appropriate primitives
        base_primitives = list(self.dsl.primitives.keys())
        primitives = self.get_3d_enhanced_primitives(base_primitives, use_3d_early)
        
        # Initialize beam
        beam = [{'grid': start_grid, 'program': [], 'score': 0.0, 'depth': 0}]
        best_solution = None
        best_score = -1
        
        start_time = time.time()
        
        for depth in range(max_depth):
            if time.time() - start_time > self.config.MAX_RUNTIME_PER_TASK:
                logger.info("Timeout reached, returning best solution")
                break
                
            new_beam = []
            beam_width = self.calculate_dynamic_beam_width(depth, max_width)
            
            for node in beam[:beam_width]:
                if time.time() - start_time > self.config.MAX_RUNTIME_PER_TASK:
                    break
                    
                for prim_name in primitives:
                    if self.should_prune_early(node, prim_name, depth):
                        continue
                        
                    new_node = self.apply_primitive(node, prim_name, target_grid)
                    if new_node:
                        new_beam.append(new_node)
                        self.stats['nodes_expanded'] += 1
                        
                        if '3d' in prim_name.lower():
                            self.stats['3d_operations'] += 1
            
            # Sort and prune beam
            new_beam.sort(key=lambda x: x['score'], reverse=True)
            beam = new_beam[:beam_width]
            
            # Check for solution
            if beam and beam[0]['score'] > best_score:
                best_solution = beam[0]
                best_score = beam[0]['score']
                
            # Early termination if perfect match
            if best_score >= 0.99:
                logger.info(f"Perfect solution found at depth {depth}")
                break
                
            # Adaptive depth adjustment
            if self.config.ADAPTIVE_DEPTH and self.should_reduce_depth(beam, depth):
                max_depth = min(max_depth, depth + 2)
                
        logger.info(f"Beam search completed: {self.stats}")
        return best_solution
    
    def analyze_task_complexity(self, start_grid, target_grid):
        complexity_factors = []
        
        # Size change complexity
        size_ratio = max(target_grid.shape[0] / start_grid.shape[0], 
                        target_grid.shape[1] / start_grid.shape[1])
        complexity_factors.append(min(size_ratio, 3.0) / 3.0)
        
        # Color change complexity
        start_palette = len(np.unique(start_grid[start_grid > 0]))
        target_palette = len(np.unique(target_grid[target_grid > 0]))
        palette_change = abs(target_palette - start_palette) / max(start_palette, 1)
        complexity_factors.append(min(palette_change, 1.0))
        
        # Structural change complexity
        start_density = np.mean(start_grid > 0)
        target_density = np.mean(target_grid > 0)
        density_change = abs(target_density - start_density)
        complexity_factors.append(density_change)
        
        return np.mean(complexity_factors)
    
    def analyze_grid_properties(self, grid):
        properties = {}
        properties['size'] = grid.shape[0] * grid.shape[1]
        properties['density'] = np.mean(grid > 0)
        properties['sparsity'] = 1.0 - properties['density']
        properties['color_variety'] = len(np.unique(grid[grid > 0]))
        
        # Check for patterns
        labeled, num_components = ndimage.label(grid > 0)
        properties['component_count'] = num_components
        if num_components > 0:
            sizes = [np.sum(labeled == i) for i in range(1, num_components + 1)]
            properties['largest_component_size'] = max(sizes)
        else:
            properties['largest_component_size'] = 0
        
        return properties
    
    def calculate_dynamic_beam_width(self, depth, max_width):
        if not self.config.DYNAMIC_BEAM_SCALING:
            return max_width
            
        # Start wide, narrow as we go deeper
        if depth == 0:
            return min(max_width * 2, 128)
        elif depth < 3:
            return max_width
        elif depth < 6:
            return max(max_width // 2, 8)
        else:
            return max(max_width // 4, 4)
    
    def should_prune_early(self, node, prim_name, depth):
        if not self.config.PRUNE_EARLY:
            return False
            
        # Don't apply the same primitive repeatedly
        if node['program'] and node['program'][-1] == prim_name:
            return True
            
        # Avoid certain sequences
        if len(node['program']) >= 2:
            last_two = node['program'][-2:]
            if last_two == ['rotate_90', 'rotate_270'] or last_two == ['rotate_270', 'rotate_90']:
                return True
                
        return False
    
    def should_reduce_depth(self, beam, current_depth):
        if len(beam) == 0:
            return True
            
        # If no improvement in recent depths, consider reducing
        if current_depth >= 6:
            recent_scores = [node['score'] for node in beam[:5]]
            if max(recent_scores) < 0.3:
                return True
                
        return False
    
    def apply_primitive(self, node, prim_name, target_grid):
        cache_key = (tuple(map(tuple, node['grid'])), prim_name)
        if cache_key in self.cache:
            self.stats['cache_hits'] += 1
            result_grid = self.cache[cache_key]
        else:
            try:
                prim_func = self.dsl.primitives[prim_name]
                result_grid = prim_func(node['grid'])
                self.cache[cache_key] = result_grid
            except Exception as e:
                logger.debug(f"Primitive {prim_name} failed: {e}")
                return None
        
        # Evaluate with novel confidence scoring
        score = calculate_iou(result_grid, target_grid)
        
        # Apply fuzzy confidence decay based on program length
        depth_penalty = self.config.CONFIDENCE_DECAY ** len(node['program'])
        final_score = score * depth_penalty
        
        return {
            'grid': result_grid,
            'program': node['program'] + [prim_name],
            'score': final_score,
            'depth': node['depth'] + 1
        }

enhanced_beam_search = EnhancedBeamSearch(enhanced_dsl, config)